## Get the lat long coordinates for each city

In [1]:
import pandas as pd
import os
import gmaps
from pprint import pprint
import requests

# Google developer API key
from gconfig import gkey

In [2]:
# import csv (merged dataframes).
fi2015 = os.path.join('data','merged_df_2015.csv')
fi2016 = os.path.join('data','merged_df_2016.csv')
fi2017 = os.path.join('data','merged_df_2017.csv')

# read csv file
df_2015 = pd.read_csv(fi2015)
df_2016 = pd.read_csv(fi2016)
df_2017 = pd.read_csv(fi2017)

In [3]:
df_2015.head()

,Unnamed: 0,region,Population,Median Age,Per Capita Income,AveragePrice,Total Volume,year
0,0,Albany,5355.093023,43.820930,31027.627907,1.538750,38749.004135,2015.0
1,1,Atlanta,11295.530547,39.880064,22475.807074,1.380577,223381.712692,2015.0
2,2,BaltimoreWashington,26444.240385,38.003846,43212.625000,1.368846,390822.880192,2015.0
3,3,Boise,26614.300000,37.280000,31414.700000,1.373750,36388.051346,2015.0
4,4,Boston,23028.012346,37.492593,42649.234568,1.473558,263990.304231,2015.0


Replace the name of the city to add state. Some cities had more than one city associated to the field, so a middle city between the two was assigned (see documentation for which city was picked)

In [4]:
replace = {'Albany': 'Albany, New York', 'Atlanta': 'Atlanta, Georgia', 'Boston': 'Boston, Massachusetts', 'HartfordSpringfield': 'Windsor Locks, Connecticut', 'New York': 'New York, New York',
          'Syracuse': 'Syracuse, New York', 'BuffaloRochester': 'Batavia, New York', 'Pittsburgh': 'Pittsburgh, Pennsylvania', 'HarrisburgScranton': 'Hazleton, Pennsylvania', 'Philadelphia': 'Philadelphia, Pennsylvania',
          'BaltimoreWashington': 'Laurel, Maryland', 'SanDiego': 'San Diego, California', 'RichmondNorfolk': 'Jamestown, Virginia', 'Roanoke': 'Roanoke, Virginia', 'RaleighGreensboro': 'Hillsborough, North Carolina',
          'Charlotte': 'Charlotte, North Carolina','SouthCarolina': 'Columbia, South Carolina', 'Atlanta': 'Atlanta, Georgia', 'Jacksonville': 'Jacksonville, Florida', 'Orlando': 'Orlando, Florida',
          'MiamiFtLauderdale': 'Miami, Florida', 'Tampa': 'Tampa, Florida', 'Nashville': 'Nashville, Tennessee', 'Louisville': 'Louisville, Kentucky', 'CincinnatiDayton': 'Monroe, Ohio',
          'Columbus': 'Columbus, Ohio', 'Indianapolis': 'Indianapolis, Indiana', 'Detroit': 'Detroit, Michigan', 'Grand Rapids': 'Grand Rapids, Michigan', 'Chicago': 'Chicago, Illinois',
          'GreatLakes': 'Milwaukee, Wisconsin', 'StLouis': 'St Louis, Missouri', 'NewOrleansMobile': 'New Orleans, Louisiana', 'DallasFtWorth': 'Dallas, Texas', 'Houston': 'Houston, Texas',
          'Denver': 'Denver, Colorado', 'Boise': 'Boise, Idaho', 'PhoenixTucson': 'Eloy, Arizona', 'WestTexNewMexico': 'Albuquerque, New Mexico', 'LasVegas': 'Las Vegas, Nevada',
          'Los Angeles': 'Los Angeles, California', 'SanFrancisco': 'San Francisco, California', 'Sacramento': 'Sacramento, California', 'California': 'Fresno, California',
          'Portland': 'Portland, Oregon', 'Seattle': 'Seattle, Washington', 'Spokane': 'Spokane, Washington'}

df_2015 = df_2015.replace({'region' : replace})
df_2016 = df_2016.replace({'region' : replace})
df_2017 = df_2017.replace({'region' : replace})

In [5]:
# citites with state stored
target_2015 = df_2015['region'].to_list()
# target_2016 = df_2016['region'].to_list()
# target_2017 = df_2017['region'].to_list()

In [6]:
# empty list to append lat long values
lat = []
lng = []

# parameters to be used in the Geocode API
params = {"address": target_2015, 
          "key": gkey}


# Build URL using the Google Maps Geocode API
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

# Execute request using the URL and dictionary containing query parameters
for city in target_2015:
    response = requests.get(base_url, params=params).json()
    lat.append(response["results"][0]["geometry"]["location"]["lat"])
    lng.append(response["results"][0]["geometry"]["location"]["lng"])


In [7]:
# Store the lat long with city in a dict
lat_long = {"region": target_2015,
           "latitude": lat,
           "longitude": lng}

In [8]:
# Make a data frame from dict
df = pd.DataFrame(lat_long)

In [9]:
# Merge the two dataframes on region to add the lat lon information
summary_df_2015 = df_2015.merge(df, on='region')
summary_df_2016 = df_2016.merge(df, on='region')
summary_df_2017 = df_2017.merge(df, on='region')
summary_df_2015.head()

,Unnamed: 0,region,Population,Median Age,Per Capita Income,AveragePrice,Total Volume,year,latitude,longitude
0,0,"Albany, New York",5355.093023,43.820930,31027.627907,1.538750,38749.004135,2015.0,42.652579,-73.756232
1,1,"Atlanta, Georgia",11295.530547,39.880064,22475.807074,1.380577,223381.712692,2015.0,42.652579,-73.756232
2,2,"Laurel, Maryland",26444.240385,38.003846,43212.625000,1.368846,390822.880192,2015.0,42.652579,-73.756232
3,3,"Boise, Idaho",26614.300000,37.280000,31414.700000,1.373750,36388.051346,2015.0,42.652579,-73.756232
4,4,"Boston, Massachusetts",23028.012346,37.492593,42649.234568,1.473558,263990.304231,2015.0,42.652579,-73.756232


### Heat Map (Age)

In [10]:
gmaps.configure(gkey)

In [11]:
# Store 'Lat' and 'Lng' into a variable 
locations_2015 = summary_df_2015[["latitude", "longitude"]]
locations_2016 = summary_df_2016[["latitude", "longitude"]]
locations_2017 = summary_df_2017[["latitude", "longitude"]]


# Store median age value in a variable 
age_2015 = summary_df_2015["Median Age"].astype(float)

In [12]:
# Create the mapping figure
fig = gmaps.figure()

# Create the heatmap layer using locations and poverty_rate
heat_layer = gmaps.heatmap_layer(locations_2015, weights=age_2015, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Heat Map (Consumption)

In [ ]:
consu_2015 = summary_df_2015["Total Volume"]
consu_layer = gmaps.symbol_layer(
    locations_2015, 
    fill_color="green",
    stroke_color='navy',
    fill_opacity=norm_count,
    stroke_opacity=norm_count,
    # the below line creates a list of strings to populate the info-box when you click on a datapoint on the map
    info_box_content=[f"Bank amount: {bank}" for bank in bank_count]
)

# Create a mapping figure
fig = gmaps.figure()

# Add the layer to the figure
fig.add_layer(consulayer)

# Display the figure
fig